In [2]:
import neuralg 
import numpy as np
import torch
import networkx
from dotmap import DotMap

07:29:16|neuralg-INFO| Initialized neuralg for cpu
07:29:16|neuralg-INFO| Setting Torch's default tensor type to Float32 (CUDA not initialized).


In [3]:
neuralg.set_precision("float64")

07:29:18|neuralg-INFO| Setting Torch's default tensor type to Float64 (CUDA not initialized).


### Spectral graph theory application

Generate $N$ random undirected graphs each with $n$ number of nodes, drawing a possble edge with probability $p$,
and compute the eigenvalues of the corresponding Adjacency and Laplacian matrices. 



In [4]:
N = 100
number_of_nodes = np.arange(5,11)
undirected_graphs = DotMap()
for n in number_of_nodes:
    for i in range(N):
        g = networkx.erdos_renyi_graph(n=n, p= 0.5)
        adjacency_matrix = torch.tensor(networkx.to_numpy_array(g))[None,:]
        laplacian_matrix = torch.tensor(networkx.laplacian_matrix(g).toarray(), dtype = torch.float64 )[None,:]
        if i == 0: 
            A = adjacency_matrix
            L = laplacian_matrix
            
        else:   
            A = torch.cat((A,adjacency_matrix),0)
            L = torch.cat((L,laplacian_matrix),0)
    undirected_graphs[str(n)].adjacency = A
    undirected_graphs[str(n)].adjacency.eigvals = neuralg.eig(A)
    undirected_graphs[str(n)].laplacian = L
    undirected_graphs[str(n)].laplacian.eigvals = neuralg.eig(L)
 

<class 'networkx.utils.decorators.argmap'> compilation 8:4: FutureWarning: laplacian_matrix will return a scipy.sparse array instead of a matrix in Networkx 3.0.


In [36]:
print(undirected_graphs["5"].adjacency.eigvals[0].sum())
print(torch.round(torch.pow(undirected_graphs["5"].adjacency.eigvals[0],10).sum()))
print(torch.pow(torch.real(torch.linalg.eigvals(undirected_graphs["5"].adjacency)[0]),10).sum())

tensor(-0.0081, grad_fn=<SumBackward0>)
tensor(45379., grad_fn=<RoundBackward>)
tensor(47674.0000)


In [12]:
neuralg.clear_loaded_models()
assert neuralg.neuralg_ModelHandler.loaded_models == {}

07:31:41|neuralg-INFO| Clearing loaded models


#### Now we can do some inference or training on these graphs based on the eigenvalue distributions
-  The spectral gap of the Laplacian of a graph is defined as the absolute difference between the two largest eigenvalues 
-  The number of cycles of length $k$ in a graph can be related to the sum of the eigenvalues to the power k. 


Lets say we want a neural network that approximates the total number of length k cycles starting from any node. In exact arithmetic, this quantity is given by 

$$trace(A^k),$$

where $A$ is the adjacency matrix of the graph, but we do not want to perform the k matrix multiplications. From the properties of the trace operator, we can relate this quantity to  the spectrum of $A$ such that  

$$ \# \text{cycles of length } k = \sum_{i} \lambda_i(A)^k $$ 

In [40]:
import torch.nn as nn 
import torch.nn.functional as F

class CycleCNN(nn.Module):
    def __init__(self, n_graph_nodes, conv_layers, filters, kernel_size):
        super(CycleCNN, self).__init__()
        self.net = []
        self.n_graph_nodes = n_graph_nodes
        self.net.append(nn.Conv2d(1,filters,kernel_size, padding = "same"))
        self.net.append(nn.BatchNorm2d(filters))
        self.net.append(nn.ReLU())
        for i in range(conv_layers-1):
            self.net.append(nn.Conv2d(filters,filters,kernel_size, padding = "same"))
            self.net.append(nn.BatchNorm2d(filters))
            self.net.append(nn.ReLU())
        
        self.net.append(nn.Conv2d(filters,1,kernel_size, padding = "same"))
        self.net.append(nn.Flatten())
        self.net.append(DenseLayer(n_graph_nodes**2,n_graph_nodes))
        self.net.append(DenseLayer(n_graph_nodes,1, is_last = True))
        self.net = nn.Sequential(*self.net)
    def forward(self, x):
        out = self.net(x)
        return out

class DenseLayer(nn.Module):

    def __init__(self, in_features, out_features, bias=True, is_last = False):
        super().__init__()
        self.is_last = is_last
        self.in_features = in_features
        self.linear = nn.Linear(in_features, out_features, bias=bias)

    def forward(self, input):
        if self.is_last: 
            return self.linear(input)
        else:
            return F.relu(self.linear(input))

In [57]:

import copy
import time

#Data parameters (Not using at the moment)
n_graph_nodes = 5
model = CycleCNN(n_graph_nodes,conv_layers= 3,filters = 32,kernel_size=3)
k = 5 #Cycle length
p = 0.5
#batch_size = 32
#Training parameters
iterations = 10000

criterion = nn.MSELoss()

lr = 3e-4 # learning rate
optimizer = torch.optim.Adam(model.parameters(), lr=lr)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 1.0, gamma=0.95)

model.train()  # turn on train mode

# ------------------------------------------------------------------------------
# A single step for debugging purposes
# ------------------------------------------------------------------------------
g = networkx.erdos_renyi_graph(n=n_graph_nodes, p= p)
adjacency_matrix = torch.tensor(networkx.to_numpy_array(g))[None,None,:]
output = model(adjacency_matrix)

print(f'==[ output:\n{output.shape}')


==[ output:
torch.Size([1, 1])


In [ ]:
def train(model: nn.Module) -> None:
    model.train()  # turn on train mode
    total_loss = 0.
    log_interval = 200
    start_time = time.time()

    for i in range(iterations):
        #Sample a batch
        A = undirected_graphs.adjacency

        #Predict # of k-cycles with model
        output = model(A)

        #Use module to compute base line 
        target_eigvals = neuralg.eig(A)

        target = torch.round(torch.pow(target_eigvals,k).sum())

        
        loss = criterion(output,target)
        

        optimizer.zero_grad()
        loss.backward()
        
        optimizer.step()

        total_loss += loss.item()
        
        if i % log_interval == 0 and i > 0:
            lr = scheduler.get_last_lr()[0]
            ms_per_batch = (time.time() - start_time) * 1000 / log_interval
            cur_loss = total_loss / log_interval
            #ppl = math.exp(cur_loss)
            print(f'| epoch {epoch:3d} | {i:5d} batches | '
                  f'lr {lr:02.4f} | ms/batch {ms_per_batch:5.2f} | '
                  f'loss {cur_loss:5.5f}')
            # print(f'| epoch {epoch:3d} | {i:5d} batches | '
            #      f'lr {lr:02.2f} | ms/batch {ms_per_batch:5.2f} | '
            #      f'loss {cur_loss:5.2f} | ppl {ppl:8.2f}')
            total_loss = 0
            start_time = time.time()

### Lets say we want to approximate the determinant of a matrix 